In [1]:
import numpy as np
from playsound import playsound
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, rfft, rfftfreq
from scipy.io.wavfile import write, read 
from scipy.io import wavfile
from time import sleep
from math import floor
import pyaudio
import wave
import random
import pandas as pd
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings("ignore")
import os
import sys
from time import sleep
from project import decoder, modulator, wav_manager

string = "MANKIND IS DEAD. BLOOD IS FUEL. HELL IS FULL"

def split_string(string, length):
    return [string[i:i+length] for i in range(0, len(string), length)]

def encode(text):
    binary_code = ''.join(format(ord(char), '08b') for char in text)
    return binary_code

def decode(binary_code):
    text = ''.join(chr(int(binary_code[i:i+8], 2)) for i in range(0, len(binary_code), 8))
    return text

print("Enter text")
text = str(input())
code = encode(text)
code_array = split_string(code, 40)
sample_rate = 44100
wava = wav_manager()
mod = modulator()
mod.set_text("11")
_, signal = mod.signal_modulation()
wava.set_writing_name("Affirmative.wav")
wava.write_sound(sample_rate, signal)
print("Sending signal to start recording")
playsound("Affirmative.wav")
sleep(1.5)
i = 1
dec = decoder()
while True:
    filename = str(i)
    chunk = 1024
    FORMAT = pyaudio.paInt16
    channels = 1
    sample_rate = 44100
    record_seconds = 0.2
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
    frames = []
    print("Trying to catch response...")
    for i in range(int(sample_rate / chunk * record_seconds)):
      data = stream.read(chunk)
      frames.append(data)
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b"".join(frames))
    wf.close()
    data = list(b"".join(frames))
    wava = wav_manager()
    wava.set_recorded_name(filename)
    data, _ = wava.get_data_sample_rate()
    xf, yf = dec.fourier_abs(data)
    yf = dec.normalize_data_array(dec.secure_split(yf, 2), xf, 1000, 1500)
    amp1, amp2 = dec.freq_sum(yf[0], xf, 1000), dec.freq_sum(yf[0], xf, 1500)
    threshold = 6
    koef = max(amp1, amp2) / min(amp1, amp2)
    print(amp2)
    if koef > threshold:
       print("Signal to start recording accepted", amp1, amp2)
       break
    amp1, amp2 = dec.freq_sum(yf[1], xf, dec.freq0), dec.freq_sum(yf[1], xf, dec.freq1)
    koef = max(amp1, amp2) / min(amp1, amp2)
    print(koef, threshold)
    if koef > threshold:
       print("Signal to start recording accepted", amp1, amp2)
       break
sleep(2)
for code in code_array:
  print("trying for", code)
  correct = False
  while not correct:
    response = False
    correct = False
    mod.set_text(code)
    _, signal = mod.signal_modulation()
    wava.set_writing_name("Encoded_signal.wav")
    wava.write_sound(sample_rate, signal)
    print("Playing signal")
    sleep(0.3)
    playsound("Encoded_signal.wav")
    sleep(0.5)
    record_seconds = 0.4
    while not response:
            p = pyaudio.PyAudio()
            stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
            frames = []
            print("Catching signal...")
            for i in range(int(sample_rate / chunk * record_seconds)):
                data = stream.read(chunk)
                frames.append(data)
            stream.stop_stream()
            stream.close()
            p.terminate()
            wf = wave.open(filename, "wb")
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(FORMAT))
            wf.setframerate(sample_rate)
            wf.writeframes(b"".join(frames))
            wf.close()
            data = list(b"".join(frames))
            wava = wav_manager()
            wava.set_recorded_name(filename)
            data, _ = wava.get_data_sample_rate()
            xf, yf = dec.fourier_abs(data)
            yf = dec.normalize_data_array(dec.secure_split(yf, 2), xf, 1000, 1500)
            amp1, amp2 = dec.freq_sum(yf[0], xf, 1000), dec.freq_sum(yf[0], xf, 1500)
            koef = amp2 / amp1
            print(koef, amp1, amp2, "koef1", "amp1")
            if koef > threshold:
                print("Signal decoded")
                sleep(1)
                correct, response = True, True
                break
            if 1 / koef > threshold:
                print("Signal decoded with error")
                correct, response = False, True
                break
            amp1, amp2 = dec.freq_sum(yf[1], xf, dec.freq0), dec.freq_sum(yf[1], xf, dec.freq1)
            koef = amp2/ amp1
            print(koef, amp1, amp2, "koef2", "amp2")
            if koef > threshold:
                print("Signal decoded")
                sleep(1)
                correct, response = True, True
                break
            if 1 / koef > threshold:
                print("Signal decoded with error")
                correct, response = False, True
                break
    if not correct:
        print("Playing the signal again")
        sleep(1)
        response = False
  

2023-06-26 16:13:57.303048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 16:13:57.346907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 16:13:57.347819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import os
import sys
os.close(sys.stderr.fileno())

In [ ]:
string = "AAA"
[string[i] for i in range(len(string))]

In [ ]:
def encode(text):
    binary = ' '.join(format(ord(char), '08b') for char in text)
    return binary

def decode(binary):
    text = ''.join(chr(int(char, 2)) for char in binary.split())
    return text
